<a href="https://colab.research.google.com/github/em-bst/enhancing-large-language-models-with-human-feedback/blob/main/RLHF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#TEST

In [ ]:
#test 2

Essai encore